input of pipeline
- article metadata
- content extractor
- skill extractor

output
- set of skills with their occurence

In [22]:
# types
from typing import TypedDict

class AricleMetadata(TypedDict):
    id: str
    title: str
    link: str
    published: str

class ParsedSkill(TypedDict):
    skill_id: str
    skill_name: str
    occurence: str

In [ ]:
import requests
import warnings

# inputs example
from extractnet import Extractor

import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor

nlp = spacy.load("en_core_web_lg")
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

In [30]:
article: AricleMetadata = {
    'id': 'https://8thlight.com/blog/shift-focus-customer-centric/',
    'title': 'Shifting Focus Beyond the Backlog to Prioritizing Customer Needs',
    'link': 'https://8thlight.com/blog/shift-focus-customer-centric/',
    'published': '2022-10-04'
}

In [32]:

def parse_content_article(
    article: AricleMetadata, 
    text_extractor: Extractor
) -> str:
    """Parse the raw text content of article."""
    link = article['link']

    raw_html = requests.get(link).text
    results = text_extractor.extract(raw_html)
    
    try :
        return results['content']
    except : 
        warnings.warn(
            f"Unable to extract content from article:\n{link}"
        )